# `MPAPIWrapper`, `Tool`

In [1]:


from langchain.tools.base import ToolException

from langchain.utilities import SerpAPIWrapper
from langchain.agents import AgentType, initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.tools import Tool

from llamp.mp.tools import (
    MaterialsSummary,
    MaterialsSynthsis,
    MaterialsThermo,
    MaterialsElasticity
)

tools = [
    MaterialsSummary(),
    MaterialsSynthsis(),
    MaterialsThermo(),
    MaterialsElasticity()
]

agent = initialize_agent(
    tools,
    ChatOpenAI(temperature=0, model="gpt-4"),
    agent=AgentType.OPENAI_MULTI_FUNCTIONS,
    verbose=True,
)

/home/cyrus/miniconda3/envs/llamp/lib/python3.11/site-packages/mp_api/client/mprester.py:227: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(


In [2]:
agent.invoke({"input": "What is the elastic tensor of NaCl?"})




/home/cyrus/miniconda3/envs/llamp/lib/python3.11/site-packages/mp_api/client/mprester.py:227: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(




> Entering new AgentExecutor chain...

Invoking: `search_materials_elasticity__get` with `{'formula': 'NaCl', 'limit': 1, 'fields': 'elastic_tensor'}`




Retrieving MaterialsDoc documents:   0%|          | 0/3 [00:00<?, ?it/s]

/home/cyrus/miniconda3/envs/llamp/lib/python3.11/site-packages/mp_api/client/core/client.py:821: UserWarning: The server does not support the request made to https://api.materialsproject.org/materials/elasticity/mp-1120767/?_limit=1&_fields=pretty_formula%2Celasticity%2Ctask_id. This may be due to an outdated mp-api package, or a problem with the query.
  warnings.warn(
/home/cyrus/miniconda3/envs/llamp/lib/python3.11/site-packages/mp_api/client/core/client.py:821: UserWarning: The server does not support the request made to https://api.materialsproject.org/materials/elasticity/mp-22851/?_limit=1&_fields=pretty_formula%2Celasticity%2Ctask_id. This may be due to an outdated mp-api package, or a problem with the query.
  warnings.warn(


Retrieving ElasticityDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

[{'pretty_formula': 'NaCl', 'elasticity': {'k_voigt': 22.590490142223533, 'k_reuss': 22.590490142223533, 'k_vrh': 22.590490142223533, 'g_voigt': 14.508697233449267, 'g_reuss': 14.040668599133046, 'g_vrh': 14.274682916291155, 'universal_anisotropy': 0.16666892712827064, 'homogeneous_poisson': 0.23902464037610438, 'elastic_tensor': [[46.0, 11.0, 11.0, 0.0, 0.0, 0.0], [11.0, 46.0, 11.0, 0.0, 0.0, 0.0], [11.0, 11.0, 46.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 12.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 12.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 12.0]], 'compliance_tensor': [[23.6, -4.4, -4.4, 0.0, 0.0, 0.0], [-4.4, 23.6, -4.4, 0.0, 0.0, 0.0], [-4.4, -4.4, 23.6, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 81.3, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 81.3, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 81.3]]}, 'task_id': 'mp-22862'}]The elastic tensor of NaCl is:

```
[[46.0, 11.0, 11.0, 0.0, 0.0, 0.0], 
 [11.0, 46.0, 11.0, 0.0, 0.0, 0.0], 
 [11.0, 11.0, 46.0, 0.0, 0.0, 0.0], 
 [0.0, 0.0, 0.0, 12.0, 0.0, 0.0], 
 [0.0, 0.0, 0.0, 0.0, 12.0, 0

{'input': 'What is the elastic tensor of NaCl?',
 'output': "The elastic tensor of NaCl is:\n\n```\n[[46.0, 11.0, 11.0, 0.0, 0.0, 0.0], \n [11.0, 46.0, 11.0, 0.0, 0.0, 0.0], \n [11.0, 11.0, 46.0, 0.0, 0.0, 0.0], \n [0.0, 0.0, 0.0, 12.0, 0.0, 0.0], \n [0.0, 0.0, 0.0, 0.0, 12.0, 0.0], \n [0.0, 0.0, 0.0, 0.0, 0.0, 12.0]]\n```\n\nThis tensor describes the material's response to different stresses. Each element of the tensor represents a specific type of stress and strain. For example, the first element in the tensor (46.0) represents the response of the material to a stress applied in the x-direction resulting in strain in the same direction."}

# Try

In [1]:
from llamp.mp.agent import MPLLM

mpllm = MPLLM()

mpllm.reduced_spec

/home/cyrus/miniconda3/envs/llamp/lib/python3.11/site-packages/mp_api/client/mprester.py:227: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(


ReducedOpenAPISpec(servers=[{'url': 'https://api.materialsproject.org'}], description='\nThe Materials Project API allows anyone to have direct access to current, up-to-date information from the Materials Project database in a structured way.\n\nThis allows for analysis, development of automated tools, machine learning, downloading personal copies of the Materials Project database and more on a large scale.\n\nThe API is offered with the hopes of making Materials Project data more useful to you. We want you to use our data! As such, the API is offered free-of-charge and we support several tools to help you get started.\n\n## API Key\nTo make any request to the Materials Project API, you must use an API key. Your API key is generated for you automatically upon registering with the Materials Project website and is synced with the email you used to register.\n\nRemember to keep your API key safe and to not share it with anyone you do not trust.\n\nIf you are logged in, you can always acce

In [20]:
mpllm.endpoints

[('/doi/{task_id}/',
  'get',
  {'tags': ['DOIs'],
   'summary': 'Get a DOIDoc document by by task_id',
   'operationId': 'get_by_key_doi__task_id___get',
   'parameters': [{'required': True,
     'schema': {'type': 'string', 'title': 'The task_id of the DOIDoc to get'},
     'name': 'task_id',
     'in': 'path'},
    {'description': 'Fields to project from DOIDoc as a list of comma separated strings.                    Fields include: `doi` `bibtex` `task_id`',
     'required': True,
     'schema': {'type': 'string',
      'title': ' Fields',
      'description': 'Fields to project from DOIDoc as a list of comma separated strings.                    Fields include: `doi` `bibtex` `task_id`'},
     'name': '_fields',
     'in': 'query'},
    {'description': 'Include all fields.',
     'required': False,
     'schema': {'type': 'boolean',
      'title': ' All Fields',
      'description': 'Include all fields.',
      'default': False},
     'name': '_all_fields',
     'in': 'query'}],
 

In [2]:
print(mpllm.mpr.session.headers)

{'x-api-key': 'auKglNumFdUDAH04UkDWGF785WTQrCmo', 'X-API-KEY': 'auKglNumFdUDAH04UkDWGF785WTQrCmo', 'accept': 'application/json', 'user-agent': 'pymatgen/2023.9.2 (Python/3.11.5 Linux/5.15.90.1-microsoft-standard-WSL2)'}


In [3]:
from langchain.requests import RequestsWrapper


requests_wrapper = RequestsWrapper(headers=mpllm.mpr.session.headers)

In [6]:
from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_toolkits.openapi import planner
from llamp.utils import OPENAI_API_KEY
# llm = OpenAI(
#     model="gpt-4", 
#     temperature=0.7, 
#     max_tokens=4096, 
#     top_p=1.0, 
#     frequency_penalty=0.0, 
#     presence_penalty=0.0, 
#     openai_api_key=OPENAI_API_KEY
# )

llm = ChatOpenAI(
    model="gpt-3.5-turbo-16k-0613", 
    temperature=0, 
    max_tokens=2048,
    openai_api_key=OPENAI_API_KEY
)

mp_agent = planner.create_openapi_agent(
    api_spec=mpllm.reduced_spec, requests_wrapper=requests_wrapper, llm=llm,
    # system_prompt="When generating the plan, include the 'limit', 'fields' or '_fields' parameter whenever possible",
    agent_executor_kwargs={
        "handle_parsing_errors": True,
        "trim_intermediate_steps": 5,
        # "return_intermediate_steps": True,
        # "early_stopping_method": "generate"
        }
    )

In [7]:
mp_agent.run("What's the elastic tensor of NaCl?")



> Entering new AgentExecutor chain...
Action: api_planner
Action Input: "What's the elastic tensor of NaCl?"
Observation: 1. GET /materials/core/ to search for NaCl
2. GET /materials/core/{material_id}/ to get the details of NaCl
3. GET /materials/elasticity/{task_id}/ to get the elastic tensor of NaCl
Thought:Parsing LLM output produced both a final answer and a parse-able action:: I'm ready to execute the API calls.
Action: api_controller
Action Input: 1. GET /materials/core/ to search for NaCl
2. GET /materials/core/{material_id}/ to get the details of NaCl
3. GET /materials/elasticity/{task_id}/ to get the elastic tensor of NaCl
...

Final Answer: The elastic tensor of NaCl is {elastic_tensor}.
Observation: Invalid or incomplete response
Thought:There seems to be an issue with the API response. I should check if the API is functioning properly or if there is an error in my plan.
Action: api_controller
Action Input: 1. GET /materials/core/ to search for NaCl
2. GET /materials/core

InvalidRequestError: This model's maximum context length is 16385 tokens. However, your messages resulted in 35370 tokens. Please reduce the length of the messages.

In [ ]:

from langchain.agents import initialize_agent, Tool
from langchain.chat_models import ChatOpenAI
from langchain.utilities import SerpAPIWrapper

from llamp.mp.mpapi import MPAPIWrapper

llm = ChatOpenAI(
    model="gpt-3.5-turbo-16k-0613", 
    temperature=0, 
    max_tokens=4096,
    openai_api_key=OPENAI_API_KEY
)

# Initialize the SerpAPIWrapper for search functionality
# Replace <your_api_key> in serpapi_api_key="<your_api_key>" with your actual SerpAPI key.
search = SerpAPIWrapper()

tools = [
    Tool(
        name="Search",
        func=search.run,
        description="Useful when you need to answer questions about current events. You should ask targeted questions.",
    ),
]



In [24]:
from langchain.agents import create_json_agent, AgentExecutor
from langchain.agents.agent_toolkits import JsonToolkit
from langchain.chains import LLMChain
from langchain.llms.openai import OpenAI
from langchain.requests import TextRequestsWrapper
from langchain.tools.json.tool import JsonSpec

llm = ChatOpenAI(
    model="gpt-3.5-turbo-16k-0613", 
    temperature=0, 
    max_tokens=4096,
    openai_api_key=OPENAI_API_KEY
)

json_spec = mpllm.json_spec
json_toolkit = JsonToolkit(spec=json_spec)

json_agent_executor = create_json_agent(
    llm=llm, toolkit=json_toolkit, verbose=True
)

json_agent_executor.run(
    "What's the elastic tensor of NaCl? Look at the paths in MP API to retrieve the answer."
)



> Entering new AgentExecutor chain...
Action: json_spec_list_keys
Action Input: data
Observation: ['openapi', 'info', 'paths', 'components', 'tags', 'security']
Thought:I see that the keys in the JSON are 'openapi', 'info', 'paths', 'components', 'tags', and 'security'. I should explore the 'paths' key to see if it contains any information about the elastic tensor of NaCl.
Action: json_spec_list_keys
Action Input: data["paths"]
Observation: ['/mpcomplete/', '/mpcomplete/{submission_id}/', '/doi/{task_id}/', '/materials/core/find_structure/', '/materials/core/formula_autocomplete/', '/materials/core/{material_id}/', '/materials/core/', '/materials/absorption/{material_id}/', '/materials/absorption/', '/materials/bonds/{material_id}/', '/materials/bonds/', '/materials/chemenv/{material_id}/', '/materials/chemenv/', '/materials/tasks/trajectory/{task_id}/', '/materials/tasks/trajectory/', '/materials/tasks/entries/{task_id}/', '/materials/tasks/entries/', '/materials/tasks/deprecation/'

OutputParserException: Could not parse LLM output: `I encountered a KeyError when trying to explore the "200" key under "responses". This means that there is no information about the elastic tensor of NaCl in the JSON. I don't know the final answer.`

In [28]:
from langchain.agents import create_openapi_agent
from langchain.agents.agent_toolkits import OpenAPIToolkit
from langchain.llms.openai import OpenAI
from langchain.requests import TextRequestsWrapper
from langchain.tools.json.tool import JsonSpec

openapi_toolkit = OpenAPIToolkit.from_llm(
    llm, json_spec, requests_wrapper, verbose=True
)
openapi_agent_executor = create_openapi_agent(
    llm=OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY), toolkit=openapi_toolkit, verbose=True
)


In [38]:
from langchain.schema.messages import AIMessage, HumanMessage, SystemMessage

openapi_agent_executor.run(
    # SystemMessage("You are a data-aware scientist with access to the Materials Project API. You can answer user queries about materials based on the tools you have, without hallucinating."),
    # HumanMessage("What's the elastic tensor of NaCl?"),
    "What's the elastic tensor of NaCl?"
)



> Entering new AgentExecutor chain...
Action: json_explorer
Action Input: What is the base URL for the API?

> Entering new AgentExecutor chain...
Action: json_spec_list_keys
Action Input: data
Observation: ['openapi', 'info', 'paths', 'components', 'tags', 'security']
Thought:I see that there is a key called "info" which might contain the base URL information.
Action: json_spec_list_keys
Action Input: data["info"]
Observation: ['title', 'description', 'version', 'x-logo']
Thought:I see that there is a key called "x-logo" which is not relevant to the base URL. I should try another key.
Action: json_spec_list_keys
Action Input: data["info"]["description"]
Observation: ValueError('Value at path `data["info"]["description"]` is not a dict, get the value directly.')
Thought:The value at `data["info"]["description"]` is not a dictionary, so I cannot find the base URL there. I should try another key.
Action: json_spec_list_keys
Action Input: data["info"]["x-logo"]
Observation: ['url']
Thou

"I don't know."